In [1]:
# Install packages if needed
!pip install tensorflow
!pip install imblearn

In [2]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io
import preprocessing as prep

In [3]:
# Prepare X, y values
target_size = (150, 150)
X, y = prep.get_X_y(percent=1, target_size=target_size)

In [4]:
# Print first 5 data rows
y[:5]

,battery,biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
X.shape

(7284, 150, 150, 3)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 5463 5463
Test Lengths: 1821 1821


In [9]:
# Create Keras Squential the model
tf.config.optimizer.set_jit(True)
base_model = tf.keras.applications.Xception(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

model = keras.Sequential()
model.add(base_model)

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12, activation='softmax'))

model.summary()

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 5, 5, 512)           │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 5, 5, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 12)                  │           6,156 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,356,468 (119.62 MB)

 Trainable params: 31,300,916 (119.40 MB)

 Non-trainable params: 55,552 (217.00 KB)

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Fit model
batch_size = 32
epochs = 10
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs, batch_size=32
)

Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.9817 - loss: 0.0796 - val_accuracy: 0.8446 - val_loss: 0.8654
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.9719 - loss: 0.1036 - val_accuracy: 0.8248 - val_loss: 0.8096
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 222s 1s/step - accuracy: 0.9772 - loss: 0.0903 - val_accuracy: 0.7985 - val_loss: 1.1009
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.9858 - loss: 0.0575 - val_accuracy: 0.8539 - val_loss: 0.8120
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9783 - loss: 0.0953 - val_accuracy: 0.8160 - val_loss: 1.0628
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.9775 - loss: 0.1010 - val_accuracy: 0.8539 - val_loss: 0.6748
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.9748 - loss: 0.0985 - val_accuracy: 0.8710 - val_loss: 0.6610
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.9828 - loss: 0.0732 - val_accu

In [16]:
# Evaluate the model using the testing data
results = model.evaluate(X_test, y_test)

for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 13s 227ms/step - accuracy: 0.8803 - loss: 0.6613
loss: 0.5776863694190979
compile_metrics: 0.8978583216667175


In [20]:
# Saving the model
with open("garbage_recycle_model.pkl", "wb") as f:
    pickle.dump(model, f)